## RLE (relative geolocation error) estimation at randomly selected points

This notebook is for calculating <B>RLE (relative geolocation error)</B> of Sentinel-1 coregistered SLCs at randomly selected points using ISCE2's Ampcor.

<b><I>Inputs</I></b>: &emsp;   Sentinel-1 coregistered SLCs produced by COMPASS     <br>
<b><I>Relative Orbit</I></b>: &emsp; 64 (ascending track)    <br>
<b><I>Period</I></b>: &emsp; 2014/12/21 - 2022/11/09 (287 SLCs)<br>
<b><I>Subswath</I></b>: &emsp; IW2 (1 burst)<br>
<b><I>Location</I></b>: &emsp; Rosamond, CA

In [ ]:
import datetime as dt
import glob
import itertools
import json
import math
import os
import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output

import folium

import geopandas as gpd

import h5py

import isce
from isce.components.mroipac.ampcor.Ampcor import Ampcor

import isceobj
from isceobj.Util.mathModule import is_power2

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

import matplotlib.pyplot as plt
from matplotlib import patches

import numpy as np

import pandas as pd

from pyproj import CRS, Proj

import requests

import scipy

import shapely.wkt as wkt
from shapely import geometry

# 0. Read First OPERA Coregistered SLC in stack

In [ ]:
# Parameters for papermill
wdir = '/u/trappist-r0/bato/work/ROSAMOND/COMPASS_TEST/A064/stack'

burst_id = 't064_135523_iw2'

In [ ]:
# define polarization/path
pol = 'VV'
burst_path = f'{wdir}/{burst_id}'

#defining dates of produced SLCs
days = glob.glob(burst_path + '/*')
days = [path.split('/')[-1] for path in days]
days = sorted(days)

# get first date
date = str(days[0])
path_h5 = f'{wdir}/{burst_id}/{date}/{burst_id}_{date}.h5'

In [ ]:
# Load the CSLC and necessary metadata
DATA_ROOT = 'science/SENTINEL1'
grid_path = f'{DATA_ROOT}/CSLC/grids'
metadata_path = f'{DATA_ROOT}/CSLC/metadata'
burstmetadata_path = f'{DATA_ROOT}/CSLC/metadata/processing_information/s1_burst_metadata'
id_path = f'{DATA_ROOT}/identification'

with h5py.File(path_h5,'r') as h5:
    cslc = h5[f'{grid_path}/{pol}'][:]
    xcoor = h5[f'{grid_path}/x_coordinates'][:]
    ycoor = h5[f'{grid_path}/y_coordinates'][:]
    dx = h5[f'{grid_path}/x_spacing'][()].astype(int)
    dy = h5[f'{grid_path}/y_spacing'][()].astype(int)
    rangePixelSize = abs(h5[f'{grid_path}/x_spacing'][()].astype(int))
    azimuthPixelSize = abs(h5[f'{grid_path}/y_spacing'][()].astype(int))
    #rangePixelSize = h5[f'{burstmetadata_path}/range_pixel_spacing'][()].astype(float)
    #azimuthPixelSize = 13.94232 #hardcoded
    epsg = h5[f'{grid_path}/projection'][()].astype(int)
    sensing_start = h5[f'{burstmetadata_path}/sensing_start'][()].astype(str)
    dims = h5[f'{burstmetadata_path}/shape'][:]
    bounding_polygon =h5[f'{id_path}/bounding_polygon'][()].astype(str) 
    orbit_direction = h5[f'{id_path}/orbit_pass_direction'][()].astype(str)


In [ ]:
# Visualize burst location on a map
cslc_poly = wkt.loads(bounding_polygon)
bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]
m = folium.Map(location=[np.mean(bbox[2:3]), np.mean(bbox[0:1])], zoom_start=9, tiles='CartoDB positron')
gdf_cslc = gpd.GeoDataFrame(index=[0], crs=f'epsg:{epsg}', geometry=[cslc_poly])
geoj_cslc = gdf_cslc.to_json()
geoj_cslc = folium.GeoJson(data=geoj_cslc,
                        style_function=lambda x: {'fillColor': 'orange'})
geoj_cslc.add_to(m)
m

In [ ]:
# Visualize CSLC
%matplotlib widget
#bbox = [xcoor.min(),xcoor.max(),ycoor.min(),ycoor.max()]
fig, ax = plt.subplots(figsize=(12, 6))
ax.imshow(20*np.log10(np.abs(cslc)), cmap='gray',interpolation=None, extent=bbox)
fig.suptitle(f'{burst_id}_{date}_{pol}')
ax.set_aspect(1)
ax.set_xlabel('Longitude (deg)')
ax.set_ylabel('Latitude (deg)')
fig.savefig('cslc.png',dpi=300,bbox_inches='tight')

### Calculating pixel location of random points

In [ ]:
#pixel location of random points 
xloc = []      
yloc = []      

#interest area
min_lat = 34.797
max_lat = 34.806
min_lon = -118.086
max_lon = -118.032

lat_ = [min_lat, max_lat]
lon_ = [min_lon, max_lon]

comb_ = list(itertools.product(lat_, lon_))

#calculating the locations in SAR image
UTMx = []
UTMy = []
xindex = []
yindex = []
xloc = []
yloc = []
_in = []
for lat, lon in comb_:
    _Proj = Proj(CRS.from_epsg(epsg))
    _x, _y = _Proj(lon, lat, inverse=False)     #conversion of lat/lon of CRs to UTM coordinates
    
    #Indices of the CRs in SLC image
    _xloc = int((_x-xcoor[0])/dx)    
    _yloc = int((_y-ycoor[0])/dy)
    
    UTMx.append(_x) 
    UTMy.append(_y)
    xindex.append(_xloc)
    yindex.append(_yloc)
    xloc.append((_x-xcoor[0])/dx)
    yloc.append((_y-ycoor[0])/dy)
    _in.append(cslc_poly.contains(geometry.Point(lon, lat)))

### Selecting random locations and calculating their pixel location in image

In [ ]:
min_x = min(xloc); max_x = max(xloc); min_y = min(yloc); max_y = max(yloc)

n_pixels = 20   #number of random pixels
randX = np.random.randint((min_x),(max_x), n_pixels)  
randY = np.random.randint((min_y),(max_y), n_pixels)

df = pd.DataFrame(data={'ID':np.arange(n_pixels),'xloc':randX,'yloc':randY})

### Offset calculation using ampcor

In [ ]:
class AmpcorPrep:
    #estimating offsets from two chips using ampcor
    
    def __init__(self, df, days, derampPath, n_neighbor, winWidth, winHeight, 
                 searchWidth, searchHeight, ovs, xstep, ystep):

        self.df = df   #pandas table
        self.days = days
        self.derampPath = derampPath 

        self.n_neighbor = n_neighbor
        self.winWidth = winWidth
        self.winHeight = winHeight
        self.xhalfwin = int(winWidth/2) + searchWidth
        self.yhalfwin = int(winHeight/2) + searchHeight

        self.searchWidth = searchWidth
        self.searchHeight = searchHeight
        self.ovs = ovs   #oversampling factor
        self.xstep = xstep
        self.ystep = ystep

        self.__check_inputs__()
        
    def __check_inputs__(self): 
        if not is_power2(self.winWidth):
            raise ValueError('Window size needs to be power of 2.') 

        if not is_power2(self.winHeight):
            raise ValueError('Window size needs to be power of 2.')
         
        if not is_power2(self.ovs):
            raise ValueError('Oversampling factor needs to be power of 2.')

        if (self.searchWidth%2 == 1):
            raise ValueError('Search window size width needs to be multiple of 2.')

        if (self.searchHeight%2 == 1):
            raise ValueError('Search window size width needs to be multiple of 2.')

    def run(self):
        ''' 
          finding correlation peak with ampcor
        '''
       
        def __create_cpxFile__(filename,dat,width,length):
            _img = isceobj.Image.createImage()
            _img.setFilename(filename)
            _img.setWidth(width)
            _img.setLength(length)
            _img.setAccessMode('write')
            _img.bands = 1
            DataType = 'CFLOAT'
            outtype = '<f'  #little endian (float)
            _img.dataType = DataType
            _img.scheme = 'BIP'
            _img.renderHdr()
            _img.renderVRT()
            _img.finalizeImage()

            fout = open(filename, "wb")
            dat_cpx = np.zeros((length,2*width))
            dat_cpx[:,::2] = np.real(dat)
            dat_cpx[:,1::2] = np.imag(dat)
            dat_cpx.astype(outtype).tofile(fout)   #little endian
            _img = None; dat_cpx = None
 
        if not os.path.isdir('./cross_correlation_random_points'):
            os.mkdir('./cross_correlation_random_points')

        for i,ref_day in enumerate(self.days):
            ref_path = f'{wdir}/{burst_id}/{ref_day}/{burst_id}_{date}.h5'
            sec_days = self.days[i+1:min(len(self.days),i+1+self.n_neighbor)]

            for sec_day in sec_days:
                sec_path = f'{wdir}/{burst_id}/{sec_day}/{burst_id}_{date}.h5'
                print(f'Ref_image: {ref_day}; Sec_image: {sec_day}')
                if os.path.isfile(f'./cross_correlation_random_points/{ref_day}_{sec_day}.csv'):
                    print(f'./cross_correlation_random_points/{ref_day}_{sec_day}.csv exist!')
                    continue
                
                dx = []
                dy = []
                snr = [] 
                for xoff, yoff in zip(self.df['xloc'],self.df['yloc']):
                    ref_slc = h5py.File(path_h5,'r')[f'{grid_path}']['VV']
                    sec_slc = h5py.File(path_h5,'r')[f'{grid_path}']['VV']

                    slccrop_ref = ref_slc[(yoff-self.yhalfwin+1):(yoff+self.yhalfwin),
                                                 (xoff-self.xhalfwin+1):(xoff+self.xhalfwin)]
                    slccrop_sec = sec_slc[(yoff-self.yhalfwin+1):(yoff+self.yhalfwin),
                                                 (xoff-self.xhalfwin+1):(xoff+self.xhalfwin)] 
                    (chiplength, chipwidth) = slccrop_ref.shape

                    objAmpcor = Ampcor()
                    objAmpcor.configure()
                    objAmpcor.setImageDataType1('complex')
                    objAmpcor.setImageDataType2('complex')
                    objAmpcor.acrossGrossOffset = 0
                    objAmpcor.downGrossOffset = 0

                    objAmpcor.windowSizeWidth = self.winWidth
                    objAmpcor.windowSizeHeight = self.winHeight
                    objAmpcor.searchWindowSizeWidth = self.searchWidth
                    objAmpcor.searchWindowSizeHeight = self.searchHeight
                    objAmpcor.oversamplingFactor = self.ovs   
                    
                    objAmpcor.setFirstPRF(1.0) 
                    objAmpcor.setSecondPRF(1.0)
                    objAmpcor.setFirstRangeSpacing(1.0)
                    objAmpcor.setSecondRangeSpacing(1.0)

                    #saving reference and secondary chips
                    refFile = 'ref.dat'; refXML = refFile + '.xml'; refVRT = refFile + '.vrt'
                    __create_cpxFile__(refFile,slccrop_ref,chipwidth,chiplength)

                    secFile = 'sec.dat'; secXML = secFile + '.xml'; secVRT = secFile + '.vrt'
                    __create_cpxFile__(secFile,slccrop_sec,chipwidth,chiplength)

                    #loading saved chips 
                    referenceImg = isceobj.createImage()   #Empty image
                    referenceImg.load(refXML) #Load from XML file
                    referenceImg.setAccessMode('read')     #Set it up for reading 
                    referenceImg.createImage()             #Create File

                    secondaryImg = isceobj.createImage()    #Empty image
                    secondaryImg.load(secXML)   #Load it from XML file
                    secondaryImg.setAccessMode('read')      #Set it up for reading
                    secondaryImg.createImage()              #Create File

                    #ampcor: amplitude cross-correlation
                    objAmpcor.ampcor(referenceImg,secondaryImg)
                    clear_output(wait=False)   #clearing cell outputs

                    referenceImg.finalizeImage()
                    secondaryImg.finalizeImage()
                    referenceImg = None
                    secondaryImg = None

                    os.remove(refFile); os.remove(refXML); os.remove(refVRT)
                    os.remove(secFile); os.remove(secXML); os.remove(secVRT)

                    offField = objAmpcor.getOffsetField()  #output of ampcor 
                    field = np.array(offField.unpackOffsets())

                    if (field.size == 0): # if ampcor fail, set the snr to 0
                        dx.append(0.0);dy.append(0.0);snr.append(0.0)
                    else:
                                   
                        success = 0
                        for item in field:
                            if (item[0]==self.xhalfwin+1) and (item[2]==self.yhalfwin+1):
                                # when the field contains the offset of the patch we needs.
                                dx.append(item[1]),dy.append(item[3]),snr.append(item[4])
                                success = 1; break
                        if success!=1:
                            # the field do not contains the offset of the patch we needs. We average the offset field.
                            dx.append(field[:,1].mean());dy.append(field[:,3].mean());
                            snr.append(field[:,4].mean())

                offset_df = pd.DataFrame({'ID':list(self.df['ID']),'dx':dx,'dy':dy,'SNR':snr})
                offset_df['dx'] = offset_df['dx']*self.xstep
                offset_df['dy'] = offset_df['dy']*self.ystep

                offset_df.to_csv(f'./cross_correlation_random_points/{ref_day}_{sec_day}.csv',index=False)

In [ ]:
#Running ampcor to find offsets
objOffset = AmpcorPrep(df, days, derampPath=burst_path,n_neighbor=3,winWidth=128,
                                winHeight=128,searchWidth=4,searchHeight=4,ovs=64,
                                xstep=rangePixelSize,ystep=azimuthPixelSize)
'''
     n_neighbor: number of neighbor pairs for calculating offsets
     winWidth, winHeight: window size for offset estimation
     searchWidth, searchHeight: search window size for offset estimation
     ovs: oversampling factor for correlation
     xstep, ystep: pixel spacing (m) in x/y
'''
objOffset.run()     #running ampcor

clear_output(wait=False)   #clearing cell outputs

In [ ]:
#reading offsets from csv files
az_off = dict(); rng_off = dict(); snr = dict()

for ID in df['ID']:
    _az_off = np.empty((len(days),len(days)))
    _az_off[:] = np.nan
    _rng_off = _az_off.copy()
    _rng_off[:] = np.nan
    
    _snr = np.zeros_like(_az_off)
    az_off[ID] = _az_off
    rng_off[ID] = _rng_off
    snr[ID] = _snr
    
for csv_file_path in glob.glob('./cross_correlation_random_points/*.csv'):
    csv_file_name = csv_file_path.split('/')[-1]
    ref_day = csv_file_name[0:8]   #reference date
    sec_day = csv_file_name[9:17]  #secondary date
    ref_i = days.index(ref_day)    #index of reference date 
    sec_i = days.index(sec_day)    #index of secondary date
    
    _df = pd.read_csv(csv_file_path)
    for ID,dxi,dyi,_snr in zip(_df['ID'],_df['dx'],_df['dy'],_df['SNR']):
        az_off[ID][ref_i,sec_i] = dyi
        rng_off[ID][ref_i,sec_i] = dxi
        snr[ID][ref_i,sec_i] = _snr

In [ ]:
#all calculated range and azimuth offsets 
all_az_off = []; all_rng_off = []; all_snr = []

for ii in az_off.keys():
    all_az_off.append(np.concatenate(az_off[ii]))
    all_rng_off.append(np.concatenate(rng_off[ii]))
    all_snr.append(np.concatenate(snr[ii]))

all_az_off = np.concatenate(all_az_off)
all_rng_off = np.concatenate(all_rng_off)
all_snr = np.concatenate(all_snr)

snr_threshold = 1.   #snr threshold

df_RLE = pd.DataFrame({'rng_off':all_rng_off, 'az_off':all_az_off, 'snr':all_snr})
df_RLE = df_RLE[(df_RLE['rng_off']!=np.nan) & (df_RLE['az_off']!=np.nan) 
                & (df_RLE['snr']>snr_threshold)].reset_index(drop=True)

#### Displaying RLEs at random points

In [ ]:
rng_rqmt = 0.5  #OPERA requirements for range offsets (0.25 m)
az_rqmt = 1.5   #OPERA requirements for azimuth offsets (0.75 m)  

fig,ax = plt.subplots(1,1,figsize=(15,15))
sc = ax.scatter(df_RLE['rng_off'],df_RLE['az_off'],s=20)
rect = patches.Rectangle((-rng_rqmt/2,-az_rqmt/2),rng_rqmt,az_rqmt,
                         linewidth=1,edgecolor='r',facecolor='none') 
ax.add_patch(rect)   #box displaying OPERA requirements
ax.grid(True)
ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.axhline(0, color='black')
ax.axvline(0, color='black')
ax.set_title('Relative geolocation error at random points', fontsize=24)
ax.set_xlabel('$\Delta$ Range (m)', fontsize=18)
ax.set_ylabel('$\Delta$ Azimuth (m)', fontsize=18)
rngMean = np.mean(df_RLE['rng_off'])
rngStd = np.std(df_RLE['rng_off'])
azMean = np.mean(df_RLE['az_off'])
azStd = np.std(df_RLE['az_off'])
ax.errorbar(rngMean,azMean,yerr=azStd,xerr=rngStd,color='w',lw=2,alpha=0.8,capsize=6)
sc_mean = ax.scatter(rngMean,azMean,alpha=0.8,c='r',s=160,marker='*')
ax.legend([sc,sc_mean,rect],['RLEs','Mean of RLEs','OPERA requirements'], fontsize=18,
          frameon=False,handletextpad=0.3)
fig.savefig('RLE.png',dpi=300,bbox_inches='tight')

In [ ]:
print('mean RLE in range: ',np.mean(df_RLE['rng_off']), 'std RLE in range: ',np.std(df_RLE['rng_off']))
print('mean RLE in azimuth: ',np.mean(df_RLE['az_off']), 'std RLE in azimuth: ',np.std(df_RLE['az_off']))